In [1]:
using CasaCore.Measures
using JLD2
using ProgressMeter
using TTCal
using BPJSpec
using Unitful
using YAML

In [2]:
function getbeam(metadata)
    ν = mean(metadata.frequencies)
    νlist = sort(collect(keys(beam_coeff)))
    idx = searchsortedlast(νlist, ν)
    if idx == length(νlist)
        return TTCal.ZernikeBeam(beam_coeff[νlist[idx]])
    else
        w1 = uconvert(NoUnits, (νlist[idx+1]-ν)/(νlist[idx+1]-νlist[idx]))
        w2 = 1 - w1
        return TTCal.ZernikeBeam(w1*beam_coeff[νlist[idx]] + w2*beam_coeff[νlist[idx+1]])
    end
end

getbeam (generic function with 1 method)

In [3]:
beam_coeff = Dict(36.528u"MHz" => [ 0.538556463745644,     -0.46866163121041965,
                                   -0.02903632892950315,   -0.008211454946665317,
                                   -0.02455123886166189,    0.010200717351278811,
                                   -0.002733004888223435,   0.012097962867146641,
                                   -0.010822907679258361],
                  41.760u"MHz" => [ 0.5683128514113496,    -0.46414332768707584,
                                   -0.049794949824191796,   0.01956938394264056,
                                   -0.028882062170310224,  -0.014311075332807512,
                                   -0.011543291444545006,   0.00665053503527859,
                                   -0.009348228819604933],
                  46.992u"MHz" => [ 0.5607524388115745,    -0.45968937134966986,
                                   -0.04003477671659007,    0.0054334058818740925,
                                   -0.029365565655034547,  -0.00022684333835518863,
                                   -0.009772599099687997,   0.007190059779729073,
                                   -0.01494324389373882],
                  52.224u"MHz" => [ 0.5648697259136155,    -0.45908927749490525,
                                   -0.03752995939112614,    0.0033934821314708244,
                                   -0.030484384773088687,   0.012225490320833442,
                                   -0.016913428790483902,  -0.004324269518531433,
                                   -0.013275940628521119],
                  57.456u"MHz" => [ 0.5647179136016398,    -0.46118768245292385,
                                   -0.029017043660228167,  -0.009711516291480747,
                                   -0.028346498468994164,   0.03494942227085211,
                                   -0.025235050863329916,  -0.011928112667488994,
                                   -0.013449331024941094],
                  62.688u"MHz" => [ 0.5634780036856724,    -0.45239381573418425,
                                   -0.020553945369180798,  -0.0038610634839508044,
                                   -0.03766765187104518,    0.034987669943576286,
                                   -0.03298552592171939,   -0.017952720352740013,
                                   -0.014260163639469253],
                  67.920u"MHz" => [ 0.554736976435005,     -0.4446983513779896,
                                   -0.019835734224238583,  -0.008902626634517375,
                                   -0.04089653832893597,    0.02106671073637622,
                                   -0.02049607316869055,    0.002052177725883946,
                                   -0.021225318022073877],
                  73.152u"MHz" => [ 0.5494343726261235,    -0.4422544222256613,
                                   -0.010377387323544141,  -0.020193950880921727,
                                   -0.03933368453654855,    0.03569618734453113,
                                   -0.020645215922528007,  -0.0007547051500611155,
                                   -0.02480903125367872])

Dict{Quantity{Float64, Dimensions:{𝐓^-1}, Units:{MHz}},Array{Float64,1}} with 8 entries:
  41.76 MHz  => [0.568313, -0.464143, -0.0497949, 0.0195694, -0.0288821, -0.014…
  46.992 MHz => [0.560752, -0.459689, -0.0400348, 0.00543341, -0.0293656, -0.00…
  67.92 MHz  => [0.554737, -0.444698, -0.0198357, -0.00890263, -0.0408965, 0.02…
  36.528 MHz => [0.538556, -0.468662, -0.0290363, -0.00821145, -0.0245512, 0.01…
  73.152 MHz => [0.549434, -0.442254, -0.0103774, -0.020194, -0.0393337, 0.0356…
  62.688 MHz => [0.563478, -0.452394, -0.0205539, -0.00386106, -0.0376677, 0.03…
  57.456 MHz => [0.564718, -0.461188, -0.029017, -0.00971152, -0.0283465, 0.034…
  52.224 MHz => [0.56487, -0.459089, -0.03753, 0.00339348, -0.0304844, 0.012225…

In [4]:
lib_directory = "/home/xhall/mmode_old/channels/14/calim-mmode-pipeline/lib/"

"/home/xhall/mmode_old/channels/14/calim-mmode-pipeline/lib/"

In [6]:
include(lib_directory * "003-calibrate.jl")

Driver

In [7]:
config_file_name = "/home/xhall/mmode_old/channels/14/calim-mmode-pipeline/projects/2017-rainy-day-power-spectrum/003-calibrate.yml"
project_file_name = "/home/xhall/mmode_old/channels/14/calim-mmode-pipeline/projects/2017-rainy-day-power-spectrum/project.yml"
wsclean_file_name = "/home/xhall/mmode_old/channels/14/calim-mmode-pipeline/projects/2017-rainy-day-power-spectrum/wsclean.yml"

"/home/xhall/mmode_old/channels/14/calim-mmode-pipeline/projects/2017-rainy-day-power-spectrum/wsclean.yml"

In [8]:
config = Driver.load(config_file_name)
wsclean = Driver.WSClean.load(wsclean_file_name)
project = Driver.Project.load(project_file_name)

Driver.Project.ProjectMetadata("2017-rainy-day-power-spectrum", "/home/xhall/mmode_old/channels/14/calim-mmode-pipeline/projects/2017-rainy-day-power-spectrum/.pipeline")

In [9]:
metadata = Driver.Project.load(project, config.metadata, "metadata")

TTCal.Metadata(Quantity{Float64, Dimensions:{𝐓^-1}, Units:{Hz}}[6.4008e7 Hz, 6.4032e7 Hz, 6.4056e7 Hz, 6.408e7 Hz, 6.4104e7 Hz, 6.4128e7 Hz, 6.4176e7 Hz, 6.42e7 Hz, 6.4224e7 Hz, 6.4296e7 Hz  …  6.6384e7 Hz, 6.6408e7 Hz, 6.6432e7 Hz, 6.6456e7 Hz, 6.648e7 Hz, 6.6504e7 Hz, 6.6528e7 Hz, 6.6552e7 Hz, 6.6576e7 Hz, 6.66e7 Hz], CasaCore.Measures.Epoch[2017-02-17T12:00:00.942, 2017-02-17T12:00:13.942, 2017-02-17T12:00:26.942, 2017-02-17T12:00:39.942, 2017-02-17T12:00:52.942, 2017-02-17T12:01:05.942, 2017-02-17T12:01:18.942, 2017-02-17T12:01:31.942, 2017-02-17T12:01:44.942, 2017-02-17T12:01:57.942  …  2017-02-18T15:58:19.022, 2017-02-18T15:58:32.022, 2017-02-18T15:58:45.022, 2017-02-18T15:58:58.022, 2017-02-18T15:59:11.022, 2017-02-18T15:59:24.022, 2017-02-18T15:59:37.022, 2017-02-18T15:59:50.022, 2017-02-18T16:00:03.022, 2017-02-18T16:00:16.022], CasaCore.Measures.Position[6371.531 km, -118d16m51s, +37d03m19s, 6371.531 km, -118d16m51s, +37d03m19s, 6371.532 km, -118d16m51s, +37d03m18s, 6371.531 

In [10]:
beam  = getbeam(metadata)

TTCal.ZernikeBeam([0.559043, -0.448489, -0.0201895, -0.00641929, -0.0393061, 0.0279238, -0.026648, -0.0078017, -0.0177945])

In [11]:
channels = 1:Nfreq(metadata)
path  = Driver.Project.workspace(project)
input = Driver.BPJSpec.load(joinpath(path, config.input))

BPJSpec.SimpleBlockArray{Complex{Float32},3,BPJSpec.MultipleFiles}("/data09/xhall_mmode_analysis/workspace/14_lmax400/2017-rainy-day-power-spectrum/002-flagged-raw-data/*", BPJSpec.Cache{Array{Complex{Float32},3}}(Base.RefValue{Bool}(false), Array{Complex{Float32},3}[]), 7756)

In [ ]:
measured = Driver.read_raw_visibilities(input, metadata, channels, config.integrations)

TTCal.Dataset{TTCal.Visibilities{TTCal.Dual,TTCal.DiagonalJonesVector}}(TTCal.Metadata(Quantity{Float64, Dimensions:{𝐓^-1}, Units:{Hz}}[6.4008e7 Hz, 6.4032e7 Hz, 6.4056e7 Hz, 6.408e7 Hz, 6.4104e7 Hz, 6.4128e7 Hz, 6.4176e7 Hz, 6.42e7 Hz, 6.4224e7 Hz, 6.4296e7 Hz  …  6.6384e7 Hz, 6.6408e7 Hz, 6.6432e7 Hz, 6.6456e7 Hz, 6.648e7 Hz, 6.6504e7 Hz, 6.6528e7 Hz, 6.6552e7 Hz, 6.6576e7 Hz, 6.66e7 Hz], CasaCore.Measures.Epoch[2017-02-17T17:46:27.958, 2017-02-17T17:46:40.958, 2017-02-17T17:46:53.958, 2017-02-17T17:47:06.958, 2017-02-17T17:47:19.958, 2017-02-17T17:47:32.958, 2017-02-17T17:47:45.958, 2017-02-17T17:47:58.958, 2017-02-17T17:48:11.958, 2017-02-17T17:48:24.958  …  2017-02-17T18:27:50.96, 2017-02-17T18:28:03.96, 2017-02-17T18:28:16.96, 2017-02-17T18:28:29.96, 2017-02-17T18:28:42.96, 2017-02-17T18:28:55.96, 2017-02-17T18:29:08.96, 2017-02-17T18:29:21.96, 2017-02-17T18:29:34.96, 2017-02-17T18:29:47.96], CasaCore.Measures.Position[6371.531 km, -118d16m51s, +37d03m19s, 6371.531 km, -118d16m51

In [13]:
sky   = readsky(config.skymodel)

TTCal.SkyModel(TTCal.Source[TTCal.Source{Array{TTCal.Gaussian{TTCal.PowerLaw},1}}("Cyg A", TTCal.Gaussian{TTCal.PowerLaw}[TTCal.Gaussian{TTCal.PowerLaw}(-60d07m30s, +40d43m58s, TTCal.PowerLaw([43170.6, 0.0, 0.0, 0.0], 1.0e6 Hz, [0.085, -0.178]), 0.000619921, 0.000108889, -1.30032), TTCal.Gaussian{TTCal.PowerLaw}(-60d08m55s, +40d44m51s, TTCal.PowerLaw([6374.46, 0.0, 0.0, 0.0], 1.0e6 Hz, [0.085, -0.178]), 0.000889279, 0.00068573, 0.758329)]), TTCal.Source{Array{TTCal.Gaussian{TTCal.PowerLaw},1}}("Cas A", TTCal.Gaussian{TTCal.PowerLaw}[TTCal.Gaussian{TTCal.PowerLaw}(-9d11m48s, +58d50m41s, TTCal.PowerLaw([1.37343e5, 0.0, 0.0, 0.0], 1.0e6 Hz, [-0.77]), 0.00101278, 0.000407728, 0.678933), TTCal.Gaussian{TTCal.PowerLaw}(-9d07m59s, +58d49m18s, TTCal.PowerLaw([1.28155e5, 0.0, 0.0, 0.0], 1.0e6 Hz, [-0.77]), 0.00111943, 0.000590988, 0.764454), TTCal.Gaussian{TTCal.PowerLaw}(-9d09m47s, +58d50m50s, TTCal.PowerLaw([106410.0, 0.0, 0.0, 0.0], 1.0e6 Hz, [-0.77]), 0.000839988, 0.000307687, 2.12759)])])

In [14]:
model    = genvis(measured.metadata, beam, sky, polarization=TTCal.Dual)

2023-01-24 10:37:36	SEVERE	MeasTable::dUTC(Double) (file /opt/src/casacore-3.5.0/measures/Measures/MeasTable.cc, line 4292)	Leap second table TAI_UTC seems out-of-date.
2023-01-24 10:37:36	SEVERE	MeasTable::dUTC(Double) (file /opt/src/casacore-3.5.0/measures/Measures/MeasTable.cc, line 4292)+	Until the table is updated (see the CASA documentation or your system admin),
2023-01-24 10:37:36	SEVERE	MeasTable::dUTC(Double) (file /opt/src/casacore-3.5.0/measures/Measures/MeasTable.cc, line 4292)+	times and coordinates derived from UTC could be wrong by 1s or more.


TTCal.Dataset{TTCal.Visibilities{TTCal.Dual,TTCal.DiagonalJonesVector}}(TTCal.Metadata(Quantity{Float64, Dimensions:{𝐓^-1}, Units:{Hz}}[6.4008e7 Hz, 6.4032e7 Hz, 6.4056e7 Hz, 6.408e7 Hz, 6.4104e7 Hz, 6.4128e7 Hz, 6.4176e7 Hz, 6.42e7 Hz, 6.4224e7 Hz, 6.4296e7 Hz  …  6.6384e7 Hz, 6.6408e7 Hz, 6.6432e7 Hz, 6.6456e7 Hz, 6.648e7 Hz, 6.6504e7 Hz, 6.6528e7 Hz, 6.6552e7 Hz, 6.6576e7 Hz, 6.66e7 Hz], CasaCore.Measures.Epoch[2017-02-17T17:46:27.958, 2017-02-17T17:46:40.958, 2017-02-17T17:46:53.958, 2017-02-17T17:47:06.958, 2017-02-17T17:47:19.958, 2017-02-17T17:47:32.958, 2017-02-17T17:47:45.958, 2017-02-17T17:47:58.958, 2017-02-17T17:48:11.958, 2017-02-17T17:48:24.958  …  2017-02-17T18:27:50.96, 2017-02-17T18:28:03.96, 2017-02-17T18:28:16.96, 2017-02-17T18:28:29.96, 2017-02-17T18:28:42.96, 2017-02-17T18:28:55.96, 2017-02-17T18:29:08.96, 2017-02-17T18:29:21.96, 2017-02-17T18:29:34.96, 2017-02-17T18:29:47.96], CasaCore.Measures.Position[6371.531 km, -118d16m51s, +37d03m19s, 6371.531 km, -118d16m51